In [ ]:
import pandas as pd
from cassandra.cluster import Cluster

Create a session connection to Cassandra cluster

In [ ]:
clstr = Cluster()
session = clstr.connect()

In [ ]:
session.execute("DROP KEYSPACE IF EXISTS songs_db")

In [ ]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

Creating keyspace songs_db

In [ ]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS songs_db WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':2
    }
""")

In [ ]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

Creating Table Users

In [ ]:
session.execute("""
CREATE TABLE IF NOT EXISTS songs_db.users ( 
    SongID INT, 
    Title TEXT, 
    Duration FLOAT, 
    Genre Text,           
    PRIMARY KEY(SongID)
);
""")

Reading Data

In [ ]:
df = pd.read_csv('exam.csv')
df

In [ ]:
for index, row in df.iterrows():
    print(f"SongID = {row.SongID}, Title ={row.Title}, Duration = {row.Duration}, Genre = {row.Genre}")

Inserting into table

In [ ]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO songs_db.users (SongID, Title, Duration, Genre)     
        VALUES ({row.SongID}, {row.Title}, {row.Duration}, {row.Genre});
        """
    )
    session.execute(f"""
        INSERT INTO songs_db.users (SongID, Title, Duration, Genre))     
        VALUES ({row.SongID}, {row.Title}, {row.Duration}, {row.Genre});
        """
    )

Using select statement

In [ ]:
rows = session.execute("select (SongID, Title, Duration, Genre) from songs_db.users ")
for row in rows:
    print(f"SongID={row[0][0]}, Title={row[0][1]}, Duration={row[0][2]}, Genre={row[0][3]}")

Using update

In [ ]:
rows = session.execute("update ( Title = 'flop') from songs_db.users where ID = 1001  ") 
for row in rows:
    print(f"SongID={row[0][0]}, Title={row[0][1]}, Duration={row[0][2]}, Genre={row[0][3]}")

Using delete

In [ ]:
rows = session.execute("Delete from songs_db.users  where ID = 1001  ")
for row in rows:
    print(f"SongID={row[0][0]}, Title={row[0][1]}, Duration={row[0][2]}, Genre={row[0][3]}")